## FitzHugh-Nagumo

We will look at a statistical inference problem using our KalmanODE solver. 

- The **FitzHugh-Nagumo** ODE model is

    $$
    \frac{dV_t}{dt} = c(V_t - V_t^3/3 + R_t), \qquad \frac{d R_t}{dt} = -(V_t - a + b R_t)/c.
    $$

- The true parameter values are $a = .2$, $b = .2$, $c = 3$.

- The initial value is $x_0=(-1,1)$. 

- The parameter priors are $\theta_j \stackrel{ind}{\sim} \mathrm{LogNormal}(\theta_{\star j}, 1)$, where $\theta_j \in \{a, b, c\}$ and $\theta_{\star j}$ is the corresponding true parameter value.

- Data is observed at time points $t = 1, 2, \ldots, 40$, with some error.  That is,

    $$
    y_{1n} \stackrel{ind}{\sim} \mathcal N(V_n, .01^2), \qquad y_{2n} \stackrel{ind}{\sim} \mathcal N(R_n, .01^2) 
    $$

- The step sizes for the discretization are $h = (.005, .01, .02, .05, .1)$.

In [ ]:
from probDE.car import car_init
from probDE.cython.KalmanODE import KalmanODE
from probDE.utils import indep_init
import numpy as np
from inference import inference

In [ ]:
def fitz(X_t, t, theta):
    "FitzHugh-Nagumo ODE function."
    a, b, c = theta
    n_state1 = len(X_t)//2
    V, R = X_t[0], X_t[n_state1] 
    return np.array([c*(V - V*V*V/3 + R), -1/c*(V - a + b*R)])

Next we will define the usual parameters required to run our solver.

In [ ]:
n_state = 6 # Total state
n_state1 = n_state2 = 3
n_meas = 2 # Total measures
state_ind = [0, 3]

# it is assumed that the solution is sought on the interval [tmin, tmax].
tmin = 0 
tmax = 40
h = 0.1 # step size
n_eval = int((tmax-tmin)/h)

# The rest of the parameters can be tuned according to ODE
# For this problem, we will use
n_var = 2
tau = [100]*n_var
sigma = [.1]*n_var

# Initial value, a, for the IVP
x0 = np.array([-1., 1.])
v0 = np.array([1, 1/3])
X0 = np.column_stack([x0, v0])
w_mat = np.array([[0.0, 1.0], [0.0, 1.0]])

# logprior parameters
theta_true = np.array([0.2, 0.2, 3]) # True theta
n_theta = len(theta_true)
phi_sd = np.ones(n_theta)

# Observation noise
gamma = 0.2

# Number of samples to draw from posterior
n_samples = 100000

First we need some data to do parameter inference. We will simulate the data using the deterministic solver **odeint**.

In [ ]:
# Initialize fitz_plot class and simulate observed data
inf = inference(state_ind, tmin, tmax, fitz)
Y_t = inf.simulate(x0, theta_true, gamma)

For a comparison, we will use the Euler's approximation method to demonstrate the effectiveness of our solver.

In [ ]:
# Euler simulation
hlst = np.array([0.1, 0.05, 0.02, 0.01, 0.005])
theta_euler = np.zeros((len(hlst), n_samples, n_theta))
for i in range(len(hlst)):
    phi_hat, phi_var = inf.phi_fit(Y_t, x0, hlst[i], theta_true, phi_sd, gamma, False)
    theta_euler[i] = inf.theta_sample(phi_hat, phi_var, n_samples)

Finally, we will our solver to do parameter inference.

In [ ]:
# Kalman simulation
theta_kalman = np.zeros((len(hlst), n_samples, n_theta))
for i in range(len(hlst)):
    kinit, x0_state = indep_init([car_init(n_state1, tau[0], sigma[0], hlst[i], w_mat[0], X0[0]),
                                  car_init(n_state2, tau[1], sigma[1], hlst[i], w_mat[1], X0[1])],
                                  n_state)
    n_eval = int((tmax-tmin)/hlst[i])
    kode = KalmanODE(n_state, n_meas, tmin, tmax, n_eval, fitz, **kinit)
    inf.kode = kode
    phi_hat, phi_var = inf.phi_fit(Y_t, x0_state, hlst[i], theta_true, phi_sd, gamma, True)
    theta_kalman[i] = inf.theta_sample(phi_hat, phi_var, n_samples)

In the plot below, we can see that the step size needs to be a lot smaller, $h=0.02$ for Euler's method to cover the true $\theta_2$, while our solver only requires $h=0.1$.

In [ ]:
# Euler, Kalman plots
inf.theta_plot(theta_euler, theta_kalman, theta_true, hlst)

## MSEIR

We will look at a multivariate ODE system called **MSEIR model**. This model is used in epidemiology where<br/>
M = Maternally-derived Immunity<br/>
S = Susceptible<br/>
E = Exposed<br/>
I = Infectious<br/>
R = Recovered<br/>

\begin{equation}
  \begin{aligned}
    \frac{dM}{dt} &= \Lambda - \delta M - \mu M \\
    \frac{dS}{dt} &= \delta M- \frac{\beta SI}{N} - \mu S \\
    \frac{dE}{dt} &= \frac{\beta SI}{N} - (\epsilon + \mu)E \\
    \frac{dI}{dt} &= \epsilon E - (\gamma + \mu)I \\
    \frac{dR}{dt} &= \gamma I - \mu R
  \end{aligned}
\end{equation}


In [ ]:
def mseir(X_t, t, theta):
    p = len(X_t)//5
    M, S, E, I, R = X_t[::p]
    N = M+S+E+I+R
    Lambda, delta, beta, mu, epsilon, gamma = theta
    dM = Lambda - delta*M - mu*M
    dS = delta*M - beta*S*I/N - mu*S
    dE = beta*S*I/N - (epsilon + mu)*E
    dI = epsilon*E - (gamma + mu)*I
    dR = gamma*I - mu*R
    return np.array([dM, dS, dE, dI, dR])

In [ ]:
# LHS Matrix of ODE
w_mat = np.array([[0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0]])

# These parameters define the order of the ODE and the CAR(p) process
n_meas = 5
n_state = 15 # number of continuous derivatives of CAR(p) solution prior
n_state1 = n_state2 = n_state3 = n_state4 = n_state5 = 3
state_ind = [0, 3, 6, 9, 12]

# it is assumed that the solution is sought on the interval [tmin, tmax].
tmin = 0
tmax = 40
h = 0.1 # step size
n_eval = int((tmax-tmin)/h)

# The rest of the parameters can be tuned according to ODE
# For this problem, we will use
tau = np.array([100, 100, 100, 100, 100])
sigma = np.array([.1, .1, .1, .1, .1])

# Initial value, x0, for the IVP
theta_true = (1.1, 0.7, 0.4, 0.005, 0.02, 0.03) # True theta
x0 = np.array([1000, 100, 50, 3, 3])
v0 = mseir(x0, 0, theta_true)
X0 = np.column_stack([x0, v0])

# logprior parameters
n_theta = len(theta_true)
phi_sd = np.ones(n_theta)

# Observation noise
gamma = 0.2

# Number of samples to draw from posterior
n_samples = 100000

In [ ]:
inf = inference(state_ind, tmin, tmax, fitz)
Y_t = inf.simulate(x0, theta_true, gamma)

In [ ]:
hlst = np.array([0.1, 0.05, 0.02, 0.01, 0.005])
theta_euler2 = np.zeros((len(hlst), n_samples, n_theta))
for i in range(len(hlst)):
    phi_hat, phi_var = inf.phi_fit(Y_t, x0, hlst[i], theta_true, phi_sd, gamma, False)
    theta_euler2[i] = inf.theta_sample(phi_hat, phi_var, n_samples)

In [ ]:
theta_kalman2 = np.zeros((len(hlst), n_samples, n_theta))
for i in range(len(hlst)):
    print(hlst[i])
    kinit, x0_state = indep_init([car_init(n_state1, tau[0], sigma[0], hlst[i], w_mat[0], X0[0]),
                                  car_init(n_state2, tau[1], sigma[1], hlst[i], w_mat[1], X0[1]),
                                  car_init(n_state3, tau[2], sigma[2], hlst[i], w_mat[2], X0[2]),
                                  car_init(n_state4, tau[3], sigma[3], hlst[i], w_mat[3], X0[3]),
                                  car_init(n_state5, tau[4], sigma[4], hlst[i], w_mat[4], X0[4])], n_state)
    n_eval = int((tmax-tmin)/hlst[i])
    kode = KalmanODE(n_state, n_meas, tmin, tmax, n_eval, mseir, **kinit)
    inf.kode = kode
    phi_hat, phi_var = inf.phi_fit(Y_t, x0_state, hlst[i], theta_true, phi_sd, gamma, True)
    theta_kalman2[i] = inf.theta_sample(phi_hat, phi_var, n_samples)

In [ ]:
inf.theta_plot(theta_euler2, theta_kalman2, theta_true, hlst)